In [ ]:
!pip install pdfplumber gradio


In [ ]:
import pdfplumber
import re

def extract_text(pdf_file):
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text


In [ ]:
def parse_resume(text):
    data = {}

    # ---------------- NAME ----------------
    name_match = re.search(r"\n([A-Z][A-Z ]{2,})\n", text)
    data["name"] = name_match.group(1).title() if name_match else "Name not found"

    # ---------------- SKILLS ----------------
    skills_match = re.search(r"SKILLS(.*?)(PROFESSIONAL SUMMARY|EXPERIENCE|EDUCATION|LINKS)", text, re.S)
    if skills_match:
        skills_text = skills_match.group(1)
        skills = re.findall(r"\b[A-Za-z ]{3,}\b", skills_text)
        data["skills"] = ", ".join(dict.fromkeys(skills[:10]))
    else:
        data["skills"] = "Skills not found"

    # ---------------- EXPERIENCE ----------------
    exp_match = re.search(r"EXPERIENCE(.*?)(EDUCATION|PROJECTS|CERTIFICATIONS|LINKS|$)", text, re.S)
    if exp_match:
        exp_text = exp_match.group(1).strip()
        data["experience"] = exp_text
    else:
        data["experience"] = "Experience not found"

    # ---------------- CGPA ----------------
    cgpa_match = re.search(r"CGPA[:\s]*([0-9.]+)", text)
    data["cgpa"] = cgpa_match.group(1) if cgpa_match else "CGPA not mentioned"

    # ---------------- GRADE ----------------
    grade_match = re.search(r"Grade[:\s]*([A-Za-z+]+)", text)
    data["grade"] = grade_match.group(1) if grade_match else "Grade not mentioned"

    return data


In [ ]:
def qa_resume(pdf, question):
    text = extract_text(pdf)
    data = extract_text_from_pdf(text)

    q = question.lower()

    if "name" in q:
        return data.get("name", "Name not found")

    elif "skill" in q:
        return f"Skills: {data.get('skills')}"

    elif "experience" in q:
        return f"Experience:\n{data.get('experience')}"

    elif "cgpa" in q:
        return f"CGPA: {data.get('cgpa')}"

    elif "grade" in q:
        if "not found" in data.get("grade", "").lower():
            return "Grade is not mentioned in the resume."
        return f"Grade: {data.get('grade')}"

    elif "course" in q or "degree" in q or "education" in q:
        if "btech" in text.lower():
            return "Course: B.Tech in Computer Science Engineering (AI & ML)"
        else:
            return "Course information not clearly mentioned."

    elif "contact" in q or "email" in q or "phone" in q:
        email = re.search(r'[\w\.-]+@[\w\.-]+', text)
        phone = re.search(r'\+?\d{10,13}', text)

        result = ""
        if phone:
            result += f"Phone: {phone.group()}\n"
        if email:
            result += f"Email: {email.group()}"

        return result if result else "Contact details not found."

    else:
        return "You can ask about name, skills, experience, course, CGPA, grade, or contact details."


In [ ]:
import gradio as gr

with gr.Blocks() as demo:

    gr.Markdown("## 📄 AI Resume Analyzer")

    pdf = gr.File(label="Upload Resume PDF")
    question = gr.Textbox(label="Ask a question")

    btn = gr.Button("Analyze")

    output = gr.Textbox(label="Answer", lines=6)

    btn.click(
        qa_resume,
        inputs=[pdf, question],
        outputs=output
    )

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0ca1008f642f877a5b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
